Графи - це спосіб репрезентації об'єктів та зв'язків між ними. 

Графи складаються з двох елементів:

* Вершини
* Ребра (зв'язки між вершинами)

In [140]:
from pyvis.network import Network

graph = Network(notebook=True, directed=True)

graph.add_nodes(list(range(4)))

graph.add_edges([(0, 1), (0, 1), (0, 2), (0, 2), (0, 3), (1, 3), (2, 3)])

graph.set_edge_smooth('dynamic')

graph.show("graph.html")


graph.html


## Naive graph representation:

In [141]:
from typing import NamedTuple, List, Tuple, TypeVar

T = TypeVar("T")

class Graph(NamedTuple):
    nodes: List[T]
    edges: List[Tuple[T, T]]
    is_directed: bool = False


In [142]:
konigsberg_nodes = ["A", "B", "C", "D"]

In [143]:
konigsberg_edges = [
    ("A", "B"),
    ("A", "B"),
    ("A", "C"),
    ("A", "C"),
    ("A", "D"),
    ("B", "D"),
    ("C", "D"),
]

In [144]:
konigsberg_bridges = Graph(konigsberg_nodes, konigsberg_edges, is_directed=False)

In [145]:
konigsberg_bridges_dir = Graph(konigsberg_nodes, konigsberg_edges, is_directed=True)

In [146]:
class WeighedGraph(Graph):
    edges: List[Tuple[T, T, float]]

In [147]:
weighted_graph = WeighedGraph(list(range(10)), [(0, 1, 0.5), 
                                                (0, 2, 0.3), 
                                                (1, 2, 0.1), 
                                                (2, 3, 0.2), 
                                                (3, 4, 0.4), 
                                                (4, 5, 0.2), 
                                                (5, 6, 0.1), 
                                                (6, 7, 0.3), 
                                                (7, 8, 0.2), 
                                                (8, 9, 0.1),
                                                (7, 0, 0.3),
                                                (6, 2, 0.1244)])

In [148]:
from pyvis.network import Network

weighted_graph_viz = Network(notebook=True)

for node in weighted_graph.nodes:
    weighted_graph_viz.add_node(node, label=str(node))

for edge in weighted_graph.edges:
    weighted_graph_viz.add_edge(edge[0], edge[1], title=edge[2], value=edge[2])

weighted_graph_viz.set_edge_smooth('dynamic')

weighted_graph_viz.show("weighted_graph.html")


weighted_graph.html


## Adjacency list (список суміжності)

Це спосіб репрезентації зв'язків у графі через створення записів у словнику виду:

Node: [Node1, Node2, Node3... NodeM]. В списку знаходяться ноди, від котрих до Node існує ребро (edge)

В Python правильний це adjacency dict. Звідси висновок: об'єкт, котрий в нас виступає в ролі ноди, має бути hashable.

Також у системах, де можливо робити декілька записів для одного і того самого ключа, ви можете розгорнути adjacency list у набір записів типу NodeM: Node1, NodeM: Node2, etc (наприклад у RDBMS)

In [149]:
def get_adjacency_list(graph: Graph):
    """
    Return the adjacency list representation
    of the graph.
    """
    adj = {node: list() for node in graph.nodes}
    for edge in graph.edges:
        node_1, node_2 = edge[0], edge[1]
        adj[node_1].append(node_2)
        if not graph.is_directed:
            adj[node_2].append(node_1)
    return adj

In [150]:
get_adjacency_list(konigsberg_bridges)

{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [151]:
get_adjacency_list(konigsberg_bridges_dir)

{'A': ['B', 'B', 'C', 'C', 'D'], 'B': ['D'], 'C': ['D'], 'D': []}

## Adjacency matrix (матриця суміжності)

In [168]:
import numpy as np

def get_adjacency_matrix(graph):
    """
    Returns the adjacency matrix of the graph.
    Important!!! This implementation assumes that nodes are labeled as integers
    """
    num_nodes = len(graph.nodes)
    adj_matrix = np.zeros((num_nodes, num_nodes), dtype=np.int8)
    
    for edge in graph.edges:
        node_1, node_2 = edge[0], edge[1]
        adj_matrix[node_1][node_2] += 1
        if not graph.is_directed:
            adj_matrix[node_2][node_1] += 1
    
    return adj_matrix

In [162]:
konigsberg_bridges_int = Graph([0, 1, 2, 3], [(0, 1), (0, 1), (0, 2), (0, 2), (0, 3), (1, 3), (2, 3)])

In [163]:
konigsberg_bridges_int_dir = Graph([0, 1, 2, 3], [(0, 1), (0, 1), (0, 2), (0, 2), (0, 3), (1, 3), (2, 3)], is_directed=True)

In [164]:
get_adjacency_matrix(konigsberg_bridges_int)

array([[0, 2, 2, 1],
       [2, 0, 0, 1],
       [2, 0, 0, 1],
       [1, 1, 1, 0]], dtype=int8)

In [165]:
get_adjacency_matrix(konigsberg_bridges_int_dir)

array([[0, 2, 2, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 0]], dtype=int8)